In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from dateutil import parser
from datetime import datetime, timedelta
import requests
import openpyxl
from pandas_market_calendars import get_calendar
from requests.adapters import HTTPAdapter
from requests.exceptions import RequestException, HTTPError, ConnectionError, Timeout

# Funções

In [2]:
def workday() -> datetime:
    return get_calendar('BMF').schedule(
        data_ini=(datetime.now() - timedelta(days=90)).strftime("%d/%m/%Y"),
        data_fim=datetime.now().strftime("%d/%m/%Y")
    ).loc[::-1, ("market_open")].iloc[0].date()

# Main

In [3]:
USER = Path(os.path.expanduser("~"))

In [4]:
excel_path = USER / r"C:\Users\GiordanoBrunoGava\OneDrive - WISIR\General\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx"
dados_cotacao__aj_consolidado_excel = pd.read_excel(excel_path)

In [5]:
dados_cotacao__aj_consolidado_excel

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
0,29/01/2024,"270,5","124,75","12,13","237,606672465","4,68",128502.0,106,8791.0,14884.0,...,"92,3441783257","8,91317598533","84,9453882851","81,6718483595","8,52076566616","8,59466865693","11,1112134366","106,050486669","113,637261909","23,0067169975"
1,30/01/2024,"270,13","123,57","12,05","237,955786001","4,6",127401.0,"104,72",8728.0,14814.0,...,"92,4369865451","8,90379369482","84,6731575127","81,5785091042","8,54879450059","8,6135580386","10,9893182681","105,45339893","113,656328564","22,7795836335"
2,31/01/2024,"266,62","123,95","11,84","231,951033185","4,58",127752.0,"106,37",8755.0,14648.0,...,"92,4833906548","8,91317598533","85,6494333864","81,5038377","8,5581374454","8,59466865693","9,58283555458","105,328203114","113,379862071","23,2338503615"
3,01/02/2024,"267,45","124,76","11,9","227,602075996","4,55",128481.0,"106,35",8800.0,14784.0,...,"91,7873290092","8,91317598533","85,6869824585","81,910081472","8,60552469592","8,65567783994","9,90163830297","104,581332165","112,273763993","23,3474170435"
4,02/02/2024,"273,01","123,32","12,23",NaN,"4,52",127182.0,"105,05",8723.0,15099.0,...,"92,5205139426","8,91317598533","85,5837225103","82,5336184244","8,60552469592","8,6461451551","10,3985955284","104,70742726","114,255122289","23,1676031303"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,18/07/2024,"344,9","124,02","15,32","212,36","5,95",127652.0,"100,3",9054.0,19761.0,...,"98,35","9,35","86,44","68,79","9,82","9,1","9,52","103,06","115,93","21,68"
120,19/07/2024,"343,76",124,"15,26","214,5","5,97",127616.0,"100,63",9003.0,19682.0,...,"98,16","9,41","87,3","68,9","9,92","9,13","9,58","104,15","116,51","21,69"
121,22/07/2024,"345,27","124,26","15,39",NaN,"6,03",127859.0,"101,8",9073.0,19788.0,...,"98,12","9,4","87,29","69,35","9,97","9,12","9,54","104,2","115,53","22,23"
122,23/07/2024,"345,5","122,95","15,38","216,91","5,93",126589.0,"99,8",8927.0,19850.0,...,"97,83","9,36","86,14","70,07","9,91","9,13","9,53","104,15","115,5","21,91"


In [6]:
dados_cotacao__aj_consolidado_excel['Data'] = pd.to_datetime(dados_cotacao__aj_consolidado_excel['Data'], format="%d/%m/%Y")

In [7]:
ultima_linha_com_data = dados_cotacao__aj_consolidado_excel['Data'].last_valid_index()

In [8]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_1=(datetime.now() - timedelta(days=180)).strftime("%d/%m/%Y")
data_fim_1=(datetime.now() - timedelta(days=120)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_1}%26data_fim%3D{data_fim_1}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_1 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [9]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_2=(datetime.now() - timedelta(days=119)).strftime("%d/%m/%Y")
data_fim_2=(datetime.now() - timedelta(days=60)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_2}%26data_fim%3D{data_fim_2}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_2 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [10]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_3=(datetime.now() - timedelta(days=59)).strftime("%d/%m/%Y")
data_fim_3=datetime.now().strftime("%d/%m/%Y") 

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_3}%26data_fim%3D{data_fim_3}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_3 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [11]:
data_1 = list()
for lin in (table := response_1.json()["tables"]["tab0"]):
    data_1.append(table[lin])

In [12]:
dados_cotacao__aj_novos_1 = pd.DataFrame(data_1)

In [13]:
data_2 = list()
for lin in (table := response_2.json()["tables"]["tab0"]):
    data_2.append(table[lin])

In [14]:
dados_cotacao__aj_novos_2 = pd.DataFrame(data_2)

dados_cotacao__aj_novos_2 = dados_cotacao__aj_novos_2.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_2.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,28/03/2024,"292,6","124,33","12,76","205,269429943","5,03",128106,109,8728,16200,...,"98,7895274227","9,04434506455","89,6819754326","80,5388285334","9,48472089623","8,82468725619","9,38116678642","107,709334659","115,531766351","22,637625281"
1,01/04/2024,"294,7","123,29","12,89",,"5,15",126990,"107,79",8712,16355,...,"99,1288437583","9,05402851751","89,5764559407","80,8000983124","9,54314833502","8,76347892226","9,39071990942","107,227303661","115,424340364","22,4104919169"
2,02/04/2024,"292,7","123,74","12,78","207,445501511","5,16",127548,"106,85",8749,16254,...,"98,2466212858","9,07339542343","89,8834217353","81,2839312364","9,53341042855","8,8023413565","9,37161366342","107,709334659","114,564932465","22,4199558071"
3,03/04/2024,"291,95","123,54","12,76","211,41832942","5,15",127318,"106,05",8753,16226,...,"98,120589504","9,07339542343","89,845051011","81,5161710399","9,53341042855","8,81205696506","9,34295429442","107,227303661","116,068896287","22,5524502694"
4,04/04/2024,"289,01","123,68","12,6",,"5,13",127427,107,8794,16079,...,"97,8782206928","9,07339542343","89,8834217353","80,7033317276","9,51393461562","8,77319453082","9,34295429442","107,030556315","115,590362344","22,5997697203"
5,05/04/2024,"292,81","122,9","12,82","207,834798718","5,11",126795,"105,9",8741,16301,...,"97,7231046537","9,11212923527","89,7491242002","80,9162182141","9,56262414794","8,88978183355","9,59133549243","107,030556315","115,434106362","22,5429863793"
6,08/04/2024,291,"125,13","12,66","207,215915979","5,1",128857,108,8874,16197,...,"97,4419568328","9,13149614119","90,075275357","80,2969120714","9,49445880269","8,82177257362","9,61044173843","107,296165232","115,971236299","22,7417280728"
7,09/04/2024,"290,2","126,21","12,68",,"5,12",129890,"109,4",8943,16118,...,"97,4807358426","9,10244578231","90,1712021678","80,3646486808","9,49445880269","8,85091939931","9,48625113943","107,591286251","115,463404359","22,8458308646"
8,10/04/2024,291,"124,15","12,74","215,710580678","5,15",128053,"106,65",8844,16286,...,"97,9169997026","9,09276232935","90,0177192705","80,5291518749","9,57236205441","8,84120379075","9,47669801643","107,610960986","115,834512315","22,1833585529"
9,11/04/2024,"294,1","123,72",13,,"5,21",127396,"105,74",8816,16520,...,"97,7231046537","9,10244578231","89,9313851407","75,4489061726","9,58209996087","8,85091939931","9,44803864743","107,237141028","115,922406305","22,0697918709"


In [15]:
data_3 = list()
for lin in (table := response_3.json()["tables"]["tab0"]):
    data_3.append(table[lin])

In [16]:
dados_cotacao__aj_novos_3 = pd.DataFrame(data_3)

dados_cotacao__aj_novos_3 = dados_cotacao__aj_novos_3.iloc[1:].reset_index(drop=True)
dados_cotacao__aj_novos_3.reset_index(drop = True)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col812,col813,col814,col815,col816,col817,col818,col819,col820,col821
0,27/05/2024,"306,12","120,87","13,6","183,34","5,85",124495,"99,62",8780,17470,...,"98,502946279","9,17095881508","87,7879047098","65,7035756815","9,58119387755","8,89721179745","9,26385169928","102,679782853","114,271134205","21,36"
1,28/05/2024,"304,61","120,18","13,59","188,95","5,85",123779,"98,4",8733,17395,...,"98,502946279","9,0926582062","87,4643259256","65,9973323537","9,45291836735","8,85797381819","9,14658775372","103,026674012","113,434523834","21,1"
2,29/05/2024,"305,45","119,12","13,59","191,45","5,84",122707,"97,35",8672,17468,...,"98,502946279","9,17095881508","88,2193430888","65,7427432378","9,57132653061","8,89721179745","9,21499172196","102,283335815","113,139249586","21,2"
3,31/05/2024,"308,2","118,66","13,59","200,14","5,81",122098,"96,87",8593,17485,...,"98,7969849246","9,20032154341","88,1507051649","65,5664892344","9,67","8,94625927152","9,54723956772","103,918679847","114,271134205","20,97"
4,03/06/2024,308,"118,44","13,57","200,66","5,81",122031,"97,5",8610,17504,...,"99,0939698492","9,24978563773","88,0036238993","65,3092105263","9,67","8,95319205298","9,42997562216","102,284931918","114,816699516","21,16"
5,04/06/2024,"311,3","118,16","13,74","201,24","5,91",121802,"96,7",8601,17734,...,"99,3414572864","9,20032154341","88,1408997472","64,1316656699","9,69","8,93338410596","9,51792358133","103,068762515","114,35048916","21,37"
6,05/06/2024,"315,58","117,9","14,06","202,71","5,96",121407,"95,8",8566,18079,...,"99,0444723618","9,23","87,8859588869","63,3202482057","9,61","8,96309602649","9,75245147245","102,731417701","115,332506719","21,43"
7,06/06/2024,"312,9","119,4","13,93",,"5,91",122898,"97,88",8646,17975,...,"98,9454773869","9,20032154341","87,8271263806","63,3301435407","9,58","8,96309602649","9,6840475042","102,880246296","116,502992295","21,77"
8,07/06/2024,"317,03","117,13","14,11","201,55","5,85",120767,"95,32",8504,18207,...,"98,5989949749","9,20032154341","88,1997322534","61,7468899522","9,62","8,98290397351","9,82085544069","102,900090108","116,979122021","21,35"
9,10/06/2024,320,"117,35","14,24",,"5,93",120759,"94,6",8474,18376,...,"97,8565326633","9,20032154341","87,7584884567","61,1531698565","9,64","9,00271192053","9,6058715405","102,49329195","115,272990503","21,13"


In [17]:
dados_cotacao__aj_novos = pd.concat([dados_cotacao__aj_novos_1, dados_cotacao__aj_novos_2, dados_cotacao__aj_novos_3], ignore_index=True) 

In [18]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.rename(
    columns = {
        # Dict comprehension para colocar os nomes antigos da coluna com os valores da primeira linha.
        nome_antigo_da_coluna: nome_novo_da_coluna for
            nome_antigo_da_coluna, nome_novo_da_coluna in
                # Zip para poder fazer unpack das duas informacoes
                zip(
                    list(dados_cotacao__aj_novos.columns), [
                        # Como os dados da primeira linha vem em formatod indeseja ja formatamos aqui
                        str(item).replace("\nPRECO_AJ(,,,A,C)", "") for
                            item in
                                # iloc para pegar a primeira linha e todas as colunas
                                list(dados_cotacao__aj_novos.iloc[0, :])
                    ]
                )
        }
)

In [19]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(index = 0)

In [20]:
# ARRUMA AS DATAS ERRADAS PARA O FORMATO DAS OUTRAS
parser.parse("12-06-2024", dayfirst = True)

datetime.datetime(2024, 6, 12, 0, 0)

In [21]:
dados_cotacao__aj_novos

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
1,29/01/2024,"270,5","124,75","12,13","237,606672465","4,68",128502,106,8791,14884,...,"92,3441783257","8,91317598533","84,9453882851","81,6718483595","8,52076566616","8,59466865693","11,1112134366","106,050486669","113,637261909","23,0067169975"
2,30/01/2024,"270,13","123,57","12,05","237,955786001","4,6",127401,"104,72",8728,14814,...,"92,4369865451","8,90379369482","84,6731575127","81,5785091042","8,54879450059","8,6135580386","10,9893182681","105,45339893","113,656328564","22,7795836335"
3,31/01/2024,"266,62","123,95","11,84","231,951033185","4,58",127752,"106,37",8755,14648,...,"92,4833906548","8,91317598533","85,6494333864","81,5038377","8,5581374454","8,59466865693","9,58283555458","105,328203114","113,379862071","23,2338503615"
4,01/02/2024,"267,45","124,76","11,9","227,602075996","4,55",128481,"106,35",8800,14784,...,"91,7873290092","8,91317598533","85,6869824585","81,910081472","8,60552469592","8,65567783994","9,90163830297","104,581332165","112,273763993","23,3474170435"
5,02/02/2024,"273,01","123,32","12,23",,"4,52",127182,"105,05",8723,15099,...,"92,5205139426","8,91317598533","85,5837225103","82,5336184244","8,60552469592","8,6461451551","10,3985955284","104,70742726","114,255122289","23,1676031303"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,19/07/2024,"343,76",124,"15,26","214,5","5,97",127616,"100,63",9003,19682,...,"98,16","9,41","87,3","68,9","9,92","9,13","9,58","104,15","116,51","21,69"
122,22/07/2024,"345,27","124,26","15,39",,"6,03",127859,"101,8",9073,19788,...,"98,12","9,4","87,29","69,35","9,97","9,12","9,54","104,2","115,53","22,23"
123,23/07/2024,"345,5","122,95","15,38","216,91","5,93",126589,"99,8",8927,19850,...,"97,83","9,36","86,14","70,07","9,91","9,13","9,53","104,15","115,5","21,91"
124,24/07/2024,"342,5","122,86","15,01",,"5,93",126422,"98,65",8908,19481,...,"97,62","9,42","85,3","69,99","9,94","9,1","9,51","104,15","115,69","21,66"


In [22]:
dados_cotacao__aj_novos.reset_index(drop = True)

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
0,29/01/2024,"270,5","124,75","12,13","237,606672465","4,68",128502,106,8791,14884,...,"92,3441783257","8,91317598533","84,9453882851","81,6718483595","8,52076566616","8,59466865693","11,1112134366","106,050486669","113,637261909","23,0067169975"
1,30/01/2024,"270,13","123,57","12,05","237,955786001","4,6",127401,"104,72",8728,14814,...,"92,4369865451","8,90379369482","84,6731575127","81,5785091042","8,54879450059","8,6135580386","10,9893182681","105,45339893","113,656328564","22,7795836335"
2,31/01/2024,"266,62","123,95","11,84","231,951033185","4,58",127752,"106,37",8755,14648,...,"92,4833906548","8,91317598533","85,6494333864","81,5038377","8,5581374454","8,59466865693","9,58283555458","105,328203114","113,379862071","23,2338503615"
3,01/02/2024,"267,45","124,76","11,9","227,602075996","4,55",128481,"106,35",8800,14784,...,"91,7873290092","8,91317598533","85,6869824585","81,910081472","8,60552469592","8,65567783994","9,90163830297","104,581332165","112,273763993","23,3474170435"
4,02/02/2024,"273,01","123,32","12,23",,"4,52",127182,"105,05",8723,15099,...,"92,5205139426","8,91317598533","85,5837225103","82,5336184244","8,60552469592","8,6461451551","10,3985955284","104,70742726","114,255122289","23,1676031303"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,19/07/2024,"343,76",124,"15,26","214,5","5,97",127616,"100,63",9003,19682,...,"98,16","9,41","87,3","68,9","9,92","9,13","9,58","104,15","116,51","21,69"
121,22/07/2024,"345,27","124,26","15,39",,"6,03",127859,"101,8",9073,19788,...,"98,12","9,4","87,29","69,35","9,97","9,12","9,54","104,2","115,53","22,23"
122,23/07/2024,"345,5","122,95","15,38","216,91","5,93",126589,"99,8",8927,19850,...,"97,83","9,36","86,14","70,07","9,91","9,13","9,53","104,15","115,5","21,91"
123,24/07/2024,"342,5","122,86","15,01",,"5,93",126422,"98,65",8908,19481,...,"97,62","9,42","85,3","69,99","9,94","9,1","9,51","104,15","115,69","21,66"


In [23]:
dados_cotacao__aj_novos

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
1,29/01/2024,"270,5","124,75","12,13","237,606672465","4,68",128502,106,8791,14884,...,"92,3441783257","8,91317598533","84,9453882851","81,6718483595","8,52076566616","8,59466865693","11,1112134366","106,050486669","113,637261909","23,0067169975"
2,30/01/2024,"270,13","123,57","12,05","237,955786001","4,6",127401,"104,72",8728,14814,...,"92,4369865451","8,90379369482","84,6731575127","81,5785091042","8,54879450059","8,6135580386","10,9893182681","105,45339893","113,656328564","22,7795836335"
3,31/01/2024,"266,62","123,95","11,84","231,951033185","4,58",127752,"106,37",8755,14648,...,"92,4833906548","8,91317598533","85,6494333864","81,5038377","8,5581374454","8,59466865693","9,58283555458","105,328203114","113,379862071","23,2338503615"
4,01/02/2024,"267,45","124,76","11,9","227,602075996","4,55",128481,"106,35",8800,14784,...,"91,7873290092","8,91317598533","85,6869824585","81,910081472","8,60552469592","8,65567783994","9,90163830297","104,581332165","112,273763993","23,3474170435"
5,02/02/2024,"273,01","123,32","12,23",,"4,52",127182,"105,05",8723,15099,...,"92,5205139426","8,91317598533","85,5837225103","82,5336184244","8,60552469592","8,6461451551","10,3985955284","104,70742726","114,255122289","23,1676031303"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,19/07/2024,"343,76",124,"15,26","214,5","5,97",127616,"100,63",9003,19682,...,"98,16","9,41","87,3","68,9","9,92","9,13","9,58","104,15","116,51","21,69"
122,22/07/2024,"345,27","124,26","15,39",,"6,03",127859,"101,8",9073,19788,...,"98,12","9,4","87,29","69,35","9,97","9,12","9,54","104,2","115,53","22,23"
123,23/07/2024,"345,5","122,95","15,38","216,91","5,93",126589,"99,8",8927,19850,...,"97,83","9,36","86,14","70,07","9,91","9,13","9,53","104,15","115,5","21,91"
124,24/07/2024,"342,5","122,86","15,01",,"5,93",126422,"98,65",8908,19481,...,"97,62","9,42","85,3","69,99","9,94","9,1","9,51","104,15","115,69","21,66"


In [24]:
if dados_cotacao__aj_consolidado_excel.index.duplicated().any():
    dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.reset_index(drop=True)
if dados_cotacao__aj_novos.index.duplicated().any():
    dados_cotacao__aj_novos = dados_cotacao__aj_novos.reset_index(drop=True)

In [25]:
dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.loc[:, ~dados_cotacao__aj_consolidado_excel.columns.duplicated()]
dados_cotacao__aj_novos = dados_cotacao__aj_novos.loc[:, ~dados_cotacao__aj_novos.columns.duplicated()]

In [26]:
# SALVA O DF CONCATENANDO NO EXCEL
output_path = USER / r'OneDrive - WISIR\General\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx'

dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(dados_cotacao__aj_novos.index[-1])
dados_cotacao__aj_novos.to_excel(output_path, index=False)

df = pd.read_excel(output_path)
df.replace(['-', '[]'], np.nan, inplace=True)
df = df.dropna(how='all', subset=df.columns.difference(['DATA']))
df.to_excel(output_path, index=False)


print('Dados atualizados e salvos no Excel.')
print('URL construída:', url + payload)

Dados atualizados e salvos no Excel.
URL construída: https://www.comdinheiro.com.br/Clientes/API/EndPoint001.phpusername=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F%26data_ini%3D27/05/2024%26data_fim%3D25/07/2024%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1%26relat_alias_automatico%3Dcmd_alias_01&format=json3
